In [1]:
!pip install scikit-learn
!pip install matplotlib
!pip install pandas
!pip install numpy

In [3]:
from tensorflow.keras.datasets import mnist 
import numpy as np

image_Path = "A_Z Handwritten Data.csv"
#przygotwanie danych i etykiet w data,labels
def load_az_dataset(image_Path):
    data=[]
    labels=[]
    for row in open(image_Path):
        #podział w wierszach na etylietę i element
        row=row.split(",")
        label = int(row[0])
        image = np.array([int(x) for x in row[1:]],dtype="uint8")
        image = image.reshape((28,28))
        
        data.append(image)
        labels.append(label)
    data= np.array(data,dtype="float32")
    labels = np.array(labels,dtype="int")
    return data,labels
def load_mnist_dataset():
    ((trainData,trainLabels),(testData,testLabels))=mnist.load_data()
    data = np.vstack([trainData,testData])
    labels= np.hstack([trainLabels,testLabels])
    return (data,labels)
    # set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
    
EPOCHS = 6
INIT_LR = 1e-1
BS=128
(azData,azLabels) = load_az_dataset(image_Path)
(digitsData,digitsLabels) = load_mnist_dataset()
#dopisanie etypiet 

azLabels +=10
data= np.vstack([azData,digitsData])
labels = np.hstack([azLabels, digitsLabels])
data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")
data = np.expand_dims(data, axis=-1)
data /= 255.0

# convertowanie opisu na wektor
le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis=0)
# account for skew in the labeled data
classTotals = labels.sum(axis=0)
classWeight = {}

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.20, stratify=labels, random_state=42)
aug = ImageDataGenerator(rotation_range=10,zoom_range=0.05,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.15,horizontal_flip=False,fill_mode="nearest")


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.regularizers import l2

class ResNet:
    @staticmethod
    def build(width, height, depth, classes, stages, filters, reg=0.0005):
        input_shape = (height, width, depth)
        input_tensor = Input(shape=input_shape)
        
        x = Conv2D(filters[0], (3, 3), padding='same', kernel_regularizer=l2(reg))(input_tensor)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        for i in range(len(stages)):
            stride = (1, 1) if i == 0 else (2, 2)

            x = ResNet.build_residual_block(x, filters[i + 1], stages[i], stride, reg=reg)

        x = AveragePooling2D(pool_size=(8, 8))(x)
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation('softmax')(x)

        model = tf.keras.models.Model(inputs=input_tensor, outputs=x)

        return model

    @staticmethod
    def build_residual_block(x, filters, blocks, stride=(1, 1), reg=0.0005):
        shortcut = x

        x = Conv2D(filters, (3, 3), strides=stride, padding='same', kernel_regularizer=l2(reg))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(reg))(x)
        x = BatchNormalization()(x)

        if stride[0] != 1 or stride[1] != 1:
            shortcut = Conv2D(filters, (1, 1), strides=stride, kernel_regularizer=l2(reg))(shortcut)
            shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        for _ in range(1, blocks):
            shortcut = x

            x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(reg))(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(reg))(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

        return x



In [5]:
from tensorflow.keras.optimizers.legacy import SGD
# initialize and compile our deep neural network
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
from keras.models import Sequential
from keras.layers import Dense, Activation
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
	(64, 64, 128, 256), reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network...")

[INFO] compiling model...


ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

In [ ]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // 2200,
	epochs=EPOCHS,
	class_weight=classWeight,
	verbose=1)


# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

# evaluate the network
predictions = model.predict(testX, batch_size=BS)
# save the model to disk
print("[INFO] serializing network...")
model.save("Models/mymodel", save_format="h5")
# construct a plot that plots and saves the training history

In [ ]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // 1300,
	epochs=1,
	class_weight=classWeight,
	verbose=1)


# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]


In [ ]:

# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

#import wymaganych bibiotek pobranych wyżej
from PIL import Image
import cv2
import numpy as np
import pytesseract
import tensorflow as tf
from tensorflow import keras
import os
model = tf.keras.models.load_model('Models/mymodel')
# initialize our list of output test images
images = []
# randomly select a few testing characters
for i in np.random.choice(np.arange(0, len(testY)), size=(49,)):
	# classify the character
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]
    # extract the image from the test data and initialize the text
    # label color as green (correct)
    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)
    # otherwise, the class label prediction is incorrect
    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)
    # merge the channels into one image, resize the image from 32x32
    # to 96x96 so we can better see it and then draw the predicted
    # label on the image
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
        color, 2)
    # add the image to our list of output images
    images.append(image)
# construct the montage for the images
montage = build_montages(images, (96, 96), (7, 7))[0]
# show the output montage
cv2.imshow("OCR Results", montage)
cv2.imwrite(' montage.jpg',montage)
cv2.waitKey(0)

In [5]:
import cv2
import numpy as np

img=cv2.imread('zadanie.png')
imgCont=img.copy()
imgFinal=img.copy()


imggray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgcanny=cv2.Canny(erode_image,94,255)
contours,_ = cv2.findContours(imgcanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

rect_d=[] #store the dimensions of my bounding boxes
min_cont_area=10
for cnt in contours:
    #print(cv2.contourArea(cnt))
    if cv2.contourArea(cnt)>min_cont_area: #Limit the contours based on area?  
        box_d=cv2.boundingRect(cnt)
        x,y,w,h=box_d
        rect_d.append([x,y,w,h]) 
        cv2.rectangle(imgCont, (x,y),(x+w,y+h), (0, 0, 255),1)

i=0
roi_n=[] #store each bounding box for later
for d in range(0,len(rect_d)):    
    x=rect_d[i][0] #Find x 
    y=rect_d[i][1] #Find y 
    w=rect_d[i][2] #Find width 
    h=rect_d[i][3] #Find height 
    roi=imgFinal[y:y+h,x:x+w]
    roi_n.append(roi)
    cv2.imwrite('Boxes/Boxed_ROIs_'+str(i) + '.png', roi)
    i+=1

cv2.imshow('Final Result',imgCont)
cv2.imwrite('Final Result.jpg',imgCont)

cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

In [6]:
import cv2
import numpy as np
    
image = cv2.imread("zadanie.png")
mask = np.zeros(image.shape, dtype=np.uint8)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)

# Create horizontal kernel then dilate to connect text contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.dilate(thresh, kernel, iterations=2)

# Find contours and filter out noise using contour approximation and area filtering
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.04 * peri, True)
    x,y,w,h = cv2.boundingRect(c)
    area = w * h
    ar = w / float(h)
    if area > 1200 and area < 50000 and ar <8:
        cv2.drawContours(mask, [c], -1, (255,255,255), -1)
        
# Bitwise-and input image and mask to get result
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
result = cv2.bitwise_and(image, image, mask=mask)
result[mask==0] = (255,255,255) # Color background white

# NEW CODE HERE TO END _____________________________________________________________
gray2 = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
thresh2 = cv2.threshold(gray2, 128, 255, cv2.THRESH_BINARY)[1]
thresh2 = 255 - thresh2
kernel = np.ones((5 ,191), np.uint8)
close = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernel)

# get external contours
contours = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

# draw contours
result2 = result.copy()
for cntr in contours:
    # get bounding boxes
    pad = 10
    x,y,w,h = cv2.boundingRect(cntr)
    cv2.rectangle( (x-pad, y-pad), (x+w+pad, y+h+pad), (0, 0, 255), 4)

cv2.imwrite("john_bboxes.jpg", result2)

cv2.imshow("thresh",thresh)
cv2.imshow("dilate",dilate)
cv2.imshow("result",result)
cv2.imshow("gray2",gray2)
cv2.imshow("thresh2",thresh2)
cv2.imshow("close",close)
cv2.imshow("result2",result2)

cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [7]:
import cv2
import numpy as np
import os

image = cv2.imread("zadanie.png")
mask = np.zeros(image.shape, dtype=np.uint8)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 9)

# Create horizontal kernel then dilate to connect text contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.dilate(thresh, kernel, iterations=2)

# Find contours and filter out noise using contour approximation and area filtering
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for i, c in enumerate(cnts):
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.04 * peri, True)
    x, y, w, h = cv2.boundingRect(c)
    area = w * h
    ar = w / float(h)
    if area > 1200 and area < 50000 and ar < 8:
        cv2.drawContours(mask, [c], -1, (255, 255, 255), -1)
        
# Bitwise-and input image and mask to get result
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
result = cv2.bitwise_and(image, image, mask=mask)
result[mask == 0] = (255, 255, 255)  # Color background white

# New code
gray2 = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
thresh2 = cv2.threshold(gray2, 128, 255, cv2.THRESH_BINARY)[1]
thresh2 = 255 - thresh2
kernel = np.ones((5, 191), np.uint8)
close = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernel)

# Get external contours
contours = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

# Draw contours and save each element
result2 = result.copy()
output_folder = "output"  # Folder to save the elements


for  cntr in contours:
    # Get bounding boxes
    pad = 10
    x, y, w, h = cv2.boundingRect(cntr)
    cv2.rectangle(result2, (x - pad, y - pad), (x + w + pad, y + h + pad), (0, 0, 255), 4)
    element = result[y - pad:y + h + pad, x - pad:x + w + pad]
    cv2.imwrite("output/wynik"+str(i)+".png", element)
    i+=1

    
cv2.imwrite("john_bboxes.jpg", result2)

cv2.imshow("thresh",thresh)
cv2.imshow("dilate",dilate)
cv2.imshow("result",result)
cv2.imshow("gray2",gray2)
cv2.imshow("thresh2",thresh2)
cv2.imshow("close",close)
cv2.imshow("result2",result2)

cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# import the necessary packages
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import time
import cv2

# load the input image and grab the image dimensions
image = cv2.imread("eroded_image.png")
orig = image.copy()
(H, W) = image.shape[:2]
# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (320,320)
rW = W / float(newW)
rH = H / float(newH)
# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet("frozen_east_text_detection.pb")
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
end = time.time()
# show timing information on text prediction
print("[INFO] text detection took {:.6f} seconds".format(end - start))

In [ ]:
# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []
# loop over the number of rows
for y in range(0, numRows):
	# extract the scores (probabilities), followed by the geometrical
	# data used to derive potential bounding box coordinates that
	# surround text
	scoresData = scores[0, 0, y]
	xData0 = geometry[0, 0, y]
	xData1 = geometry[0, 1, y]
	xData2 = geometry[0, 2, y]
	xData3 = geometry[0, 3, y]
	anglesData = geometry[0, 4, y]
    
    # loop over the number of columns
	for x in range(0, numCols):
		# if our score does not have sufficient probability, ignore it
		if scoresData[x] < 0.5:
			continue
		# compute the offset factor as our resulting feature maps will
		# be 4x smaller than the input image
		(offsetX, offsetY) = (x * 4.0, y * 4.0)
		# extract the rotation angle for the prediction and then
		# compute the sin and cosine
		angle = anglesData[x]
		cos = np.cos(angle)
		sin = np.sin(angle)
		# use the geometry volume to derive the width and height of
		# the bounding box
		h = xData0[x] + xData2[x]
		w = xData1[x] + xData3[x]
		# compute both the starting and ending (x, y)-coordinates for
		# the text prediction bounding box
		endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
		endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
		startX = int(endX - w)
		startY = int(endY - h)
		# add the bounding box coordinates and probability score to
		# our respective lists
		rects.append((startX, startY, endX, endY))
		confidences.append(scoresData[x])
# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)
# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)
	# draw the bounding box on the image
	cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
# show the output image
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)

In [ ]:
# importuj potrzebne biblioteki
from tensorflow.keras.models import load_model
import numpy as np
import cv2


# załaduj model OCR do rozpoznawania pisma odręcznego
print("[INFO] wczytywanie modelu OCR do rozpoznawania pisma odręcznego...")
model = tf.keras.models.load_model('Models/mymodel')
# load the input image from disk, convert it to grayscale, and blur
# it to reduce noise
image = cv2.imread("zadanie.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
# initialize the list of contour bounding boxes and associated
# characters that we'll be OCR'ing
chars = []
# loop over the contours
for c in cnts:
	# compute the bounding box of the contour
	(x, y, w, h) = cv2.boundingRect(c)
	# filter out bounding boxes, ensuring they are neither too small
	# nor too large
	if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):
		# extract the character and threshold it to make the character
		# appear as *white* (foreground) on a *black* background, then
		# grab the width and height of the thresholded image
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,
			cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape
		# if the width is greater than the height, resize along the
		# width dimension
		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		# otherwise, resize along the height
		else:
			thresh = imutils.resize(thresh, height=32)
            # re-grab the image dimensions (now that its been resized)
		# and then determine how much we need to pad the width and
		# height such that our image will be 32x32
		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)
		# pad the image and force 32x32 dimensions
		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(0, 0, 0))
		padded = cv2.resize(padded, (32, 32))
		# prepare the padded image for classification via our
		# handwriting OCR model
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)
		# update our list of characters that will be OCR'd
		chars.append((padded, (x, y, w, h)))
        # OCR the characters using our handwriting recognition model
preds = model.predict(chars)
# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [8]:
import pytesseract
from PIL import Image
image_path ="ilustracja_pytania.jpg"
# Konfiguracja pytesseracta
custom_config = r'--oem 3 --psm 6 -l pol'
# Możesz dostosować powyższą konfigurację w zależności od swoich potrzeb

# Ścieżka do pliku obrazu z tekstem

# Wczytanie obrazu
image = Image.open(image_path)

# Wykorzystanie pytesseracta do rozpoznania tekstu
recognized_text = pytesseract.image_to_string(image, config=custom_config)

# Wyświetlenie rozpoznanego tekstu
print(recognized_text)

ELWAB SA
UL. SMOLEŃSKA 64
80-410 GDAŃSK
Tel: 0483414002 Fax: 0483410350
NIP. 684-001-04-59
dn.14r12.21 wydr.1129
PARAGON FISKALNY
WODA MINERAL. 1,5L 3*2,00=6,00C
PIZZA 1*5,00=5,00A ' 1
JABŁKA 1,84 * 2,99=5,50 B
RABAT 105 NA JABŁKA -0,55
PODSUMA 1595 ) 2
RAZEM DO OBNIŻKA ŚWIĄTECZNA 16,50
RABAT OBNIŻKA ŚWIĄTECZNA 165 || 3
RAZEM DO PROM. 3 WCENIE 2 6,00
RABAT PROM. 3 W CENIE 2 -2,00
Sp.op.A 4,50 PTU A=22,00% 0,81
Sp.op.B 4,50 PTU B= 7,00% 029 [ 4
Sp.op.C 3,40 PTU C= 0,00% 0,00
Razem PTU 1,10
SUMA PLN 12,30 |) 5
ZAPŁACONO GOTÓWKĄ PLN 12,30
0036/0041 0130 SZEF 9:48
PL BAQ 00000000

